In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install mne

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 16.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
import mne  # For EEG data preprocessing and feature extraction
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt


In [ ]:
import mne
import os

# Specify the folder path (replace with your actual path)
folder_path = "/content/drive/MyDrive/EEG DATASET"

# Error handling: Check if folder exists
if not os.path.exists(folder_path):
    raise ValueError(f"Folder '{folder_path}' does not exist. Please provide a valid path.")

# Create an empty list to store the files (optional)
files = []

# Iterate through files in the folder using os.listdir()
for filename in os.listdir(folder_path):
    # Check if it's a file (not a directory) using os.path.isfile()
    if os.path.isfile(os.path.join(folder_path, filename)):
        # Construct the full path to the file
        full_path = os.path.join(folder_path, filename)

        # Load the file using mne.io.read() (replace with specific MNE function if needed)
        data = mne.io.read_raw_cnt(full_path)

        # Process the data (replace with your specific operations)
        print(f"Processing file: {filename}")
        # ... your data processing logic here ...

        # Optionally, add the file to the list (if needed)
        files.append(data)




<ipython-input-4-15bfd2a69edd>:22: RuntimeWarning:   Could not parse meas date from the header. Setting to None.
  data = mne.io.read_raw_cnt(full_path)


Processing file: 2_2_20180419.cnt
Processing file: 1_2_20180810.cnt


<ipython-input-4-15bfd2a69edd>:22: RuntimeWarning:   Could not parse meas date from the header. Setting to None.
  data = mne.io.read_raw_cnt(full_path)


Processing file: 3_2_20180419.cnt


<ipython-input-4-15bfd2a69edd>:22: RuntimeWarning:   Could not parse meas date from the header. Setting to None.
  data = mne.io.read_raw_cnt(full_path)


...

Processing file: 10_2_20180524.cnt
Processing file: 11_2_20180508.cnt
Processing file: 12_2_20180508.cnt
Processing file: 13_2_20180806.cnt


<ipython-input-4-15bfd2a69edd>:22: RuntimeWarning:   Could not parse meas date from the header. Setting to None.
  data = mne.io.read_raw_cnt(full_path)


Processing file: 14_2_20180423.cnt
Processing file: 15_2_20180807.cnt


<ipython-input-4-15bfd2a69edd>:22: RuntimeWarning:   Could not parse meas date from the header. Setting to None.
  data = mne.io.read_raw_cnt(full_path)


Processing file: 16_2_20180815.cnt


In [ ]:
for file in files:
  print(file.info)

In [ ]:
# check channel names
channels = []
def channel(raw):
  ch_names = raw.ch_names
  # drop non-used channels
  useless_ch = ['M1', 'M2', 'VEO', 'HEO']
  raw = raw.drop_channels(useless_ch)
  return raw

for file in files:
  raw = channel(file)
  channels.append(raw)




In [ ]:
filter = []
def pre(raw):
  raw.load_data()
  raw = raw.filter(l_freq=0.5, h_freq=30)
  print("Original sampling frequency:", raw.info['sfreq'], "Hz")

  # Define the new sampling frequency after downsampling
  new_sfreq = 200  # for example, downsampling to 250 Hz

  # Downsampling the data
  raw = raw.resample(sfreq=new_sfreq)

  return raw

for file in channels:
  raw = pre(file)
  filter.append(raw)



In [ ]:
ref_filter = []
def filter2(raw):
  raw = raw.set_eeg_reference(ref_channels='average')
  return raw

for file in filter:
  raw = filter2(file)
  ref_filter.append(raw)

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
E

In [ ]:
for i, raw in enumerate(ref_filter):
    filename = f'/content/drive/MyDrive/filter_eeg_1/ss_{i+1}_2.fif'
    raw.save(filename)


In [ ]:
data_files = []
for file in ref_filter:
  raw = file.get_data()
  data_files.append(raw)

In [ ]:
import numpy as np
base = []

def apply_reference_and_baseline_correction(data, start_seconds, end_seconds, pre_stim_window, sampling_rate):

  # Apply common average reference (CAR)


  # Perform trial-based baseline correction
  corrected_data = data.copy()
  for trial_idx, (start, end) in enumerate(zip(start_seconds, end_seconds)):
    # Calculate baseline window in samples (relative to trial start)
    baseline_start = int(max(0, (start - pre_stim_window) * sampling_rate))
    baseline_end = int(start * sampling_rate)

    # Calculate baseline for each channel
    baseline = np.mean(corrected_data[:, baseline_start:baseline_end], axis=1).reshape(-1, 1)

    # Apply baseline correction for the entire trial
    corrected_data[:, start_seconds[trial_idx]*sampling_rate:end_seconds[trial_idx]*sampling_rate] -= baseline
    corrected_data = (corrected_data - np.mean(corrected_data, axis=0)) / np.std(corrected_data, axis=0)

  return corrected_data

# Replace with your actual data
start_seconds = [30, 132, 287, 555, 773, 982, 1271, 1628, 1730, 2025, 2227, 2435, 2667, 2932, 3204]
end_seconds = [102, 228, 524, 742, 920, 1240, 1568, 1697, 1994, 2166, 2401, 2607, 2901, 3172, 3359]
pre_stim_window = 0.2
sampling_rate = 200  # Hz (assuming you know your sampling rate)

# Apply referencing and baseline correction
for data in data_files:
  raw = apply_reference_and_baseline_correction(data, start_seconds, end_seconds, pre_stim_window, sampling_rate)
  base.append(raw)





In [ ]:
import os
import numpy as np

def save(f, segment):# Directory to save CSV files
  save_dir = "/content/drive/MyDrive/Thesis/data"
  os.makedirs(save_dir, exist_ok=True)

  # Save each trial into a separate CSV file
  for i, segment in enumerate(segment):
      csv_file = os.path.join(save_dir, f"ss_{f}_{i+1}.csv")
      np.savetxt(csv_file, segment, delimiter=",")
      print(f"Saved file: {csv_file}")
for gi in range(len(segment)):
  save(gi, segment[gi])

Saved file: /content/drive/MyDrive/Thesis/data/ss_0_1.csv
Saved file: /content/drive/MyDrive/Thesis/data/ss_0_2.csv
Saved file: /content/drive/MyDrive/Thesis/data/ss_0_3.csv
Saved file: /content/drive/MyDrive/Thesis/data/ss_0_4.csv
Saved file: /content/drive/MyDrive/Thesis/data/ss_0_5.csv
Saved file: /content/drive/MyDrive/Thesis/data/ss_0_6.csv
Saved file: /content/drive/MyDrive/Thesis/data/ss_0_7.csv
Saved file: /content/drive/MyDrive/Thesis/data/ss_0_8.csv
Saved file: /content/drive/MyDrive/Thesis/data/ss_0_9.csv
Saved file: /content/drive/MyDrive/Thesis/data/ss_0_10.csv
Saved file: /content/drive/MyDrive/Thesis/data/ss_0_11.csv
Saved file: /content/drive/MyDrive/Thesis/data/ss_0_12.csv
Saved file: /content/drive/MyDrive/Thesis/data/ss_0_13.csv
Saved file: /content/drive/MyDrive/Thesis/data/ss_0_14.csv
Saved file: /content/drive/MyDrive/Thesis/data/ss_0_15.csv
Saved file: /content/drive/MyDrive/Thesis/data/ss_1_1.csv
Saved file: /content/drive/MyDrive/Thesis/data/ss_1_2.csv
Saved fi

In [ ]:
#Zero-Padding
import numpy as np

max_samples = max(trial.shape[1] for trial in trials)

sample = np.zeros((len(trials), 62, max_samples))

for i, trial in enumerate(trials):
  padding_length = max_samples - trial.shape[1]
  sample[i, :, :] = np.pad(trial, ((0, 0), (0, padding_length)), mode='constant', constant_values=0)

print(sample.shape)

(240, 62, 59400)


In [ ]:
def average_pooling(segment):

  # Convert segment to tensor (assuming data is a NumPy array)
  segment_tensor = tf.convert_to_tensor(segment, dtype=tf.float32)
  # Apply average pooling
  segment_tensor = tf.reshape(segment_tensor, [segment_tensor.shape[0], segment_tensor.shape[1], 1])
  pooled_segment = tf.keras.layers.AveragePooling2D(3,3)(
      tf.expand_dims(segment_tensor, axis=0))
  # Squeeze the dimension

  return tf.squeeze(pooled_segment, axis=0)

# Apply pooling to each data segment
pooled_data = [average_pooling(segment) for segment in trials]


In [ ]:
pooled = []
for i in pooled_data:
  raw = tf.reshape(i,[i.shape[0], i.shape[1]])
  pooled.append(raw)

In [ ]:
#Average Pooling
import numpy as np


max_samples = max(trial.shape[1] for trial in pooled)

sample = np.zeros((len(pooled), 20, max_samples))

for i, trial in enumerate(pooled):
  padding_length = max_samples - trial.shape[1]
  sample[i, :, :] = np.pad(trial, ((0, 0), (0, padding_length)), mode='constant', constant_values=0)

print(sample.shape)

(240, 20, 19800)


In [ ]:
import os
import numpy as np

def save(i, segment):# Directory to save CSV files
  save_dir = "/content/drive/MyDrive/pooled_data"
  os.makedirs(save_dir, exist_ok=True)


  csv_file = os.path.join(save_dir, f"s{i+1}.csv")
  np.savetxt(csv_file, segment, delimiter=",")
  print(f"Saved file: {csv_file}")
for i, segment in enumerate(sample):
  save(i, segment)
  print(segment.shape)